In [1]:
using Distributions
mean = [2.,3.]
C = [0.2 0; 0 0.3]
d = MvNormal(mean, C)
x = rand(d, 2000)

2×2000 Matrix{Float64}:
 1.5258   2.22738  1.53614  2.28665  …  2.15072  1.96453  2.4128   1.72141
 2.96187  2.90154  3.27173  3.2597      2.99756  2.68633  3.31022  2.89368

In [ ]:
import vedo
import numpy as np
import time
import imageio # Import imageio for video creation
import os      # Import os for directory operations

# Initialize the plotter
# 'axes' sets up a standard 3D axis system for context
# 'bg' sets the background color
# 'bg2' sets a secondary background color for a gradient effect
plotter = vedo.Plotter(axes=7, bg='lightgrey', bg2='lightblue', size=(900, 900))

# Create a sphere actor
# 'r' is the radius, 'res' is the resolution (number of segments)
# 'c' is the color, 'alpha' is the transparency
sphere = vedo.Sphere(r=1, res=64, c='blue', alpha=0.6)

# Add the sphere to the plotter
plotter.add(sphere)

# --- Add Latitude Lines ---
# Number of latitude lines to draw
num_latitudes = 10
for i in range(num_latitudes):
    # Calculate latitude angle (from -90 to 90 degrees, excluding poles)
    lat_angle = -90 + (180 / (num_latitudes + 1)) * (i + 1)
    # Convert to radians
    lat_rad = np.deg2rad(lat_angle)

    # Calculate radius of the latitude circle at this angle
    # This is based on the sphere's radius (r=1) and the latitude angle
    radius_at_lat = np.cos(lat_rad) * sphere.r

    # Calculate the z-coordinate for this latitude line
    # This is based on the sphere's radius (r=1) and the latitude angle
    z_coord = np.sin(lat_rad) * sphere.r

    # Create points for the latitude circle
    # 'ns' is the number of points in the circle for smoothness
    latitude_points = []
    ns = 100
    for j in range(ns + 1):
        angle = 2 * np.pi * j / ns
        x = radius_at_lat * np.cos(angle)
        y = radius_at_lat * np.sin(angle)
        latitude_points.append([x, y, z_coord])

    # Create a line actor for the latitude and add it to the plotter
    # 'c' is color, 'lw' is line width
    latitude_line = vedo.Line(latitude_points, c='red', lw=2)
    plotter.add(latitude_line)

# --- Add Longitude Lines ---
# Number of longitude lines to draw
num_longitudes = 12
for i in range(num_longitudes):
    # Calculate longitude angle (from 0 to 360 degrees)
    lon_angle = (360 / num_longitudes) * i
    # Convert to radians
    lon_rad = np.deg2rad(lon_angle)

    # Create points for the longitude half-circle (from south pole to north pole)
    longitude_points = []
    ns = 100 # Number of points for smoothness
    for j in range(ns + 1):
        # Parametric equation for a circle in the x-z plane, then rotated
        theta = np.pi * j / ns - np.pi / 2 # Angle from -pi/2 to pi/2 (south pole to north pole)
        x = sphere.r * np.cos(theta) * np.cos(lon_rad)
        y = sphere.r * np.cos(theta) * np.sin(lon_rad)
        z = sphere.r * np.sin(theta)
        longitude_points.append([x, y, z])

    # Create a line actor for the longitude and add it to the plotter
    longitude_line = vedo.Line(longitude_points, c='yellow', lw=2)
    plotter.add(longitude_line)

# Set initial camera position for a good view
plotter.camera.SetPosition([5, 5, 5])
plotter.camera.SetFocalPoint([0, 0, 0])
plotter.camera.SetViewUp([0, 0, 1])

# --- Video Saving Setup ---
# Directory to save individual frames
frames_dir = "sphere_frames"
if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)

# Maximum number of frames to capture for the video
max_frames = 100
frame_count = 0
video_filename = "spinning_sphere.mp4"

# --- Animation Loop ---
# Start the interactive window, but do not block execution
# 'interactive=False' means the window opens but the script continues
plotter.show(interactive=False)

# Rotation speed (degrees per frame)
rotation_speed = 1.0

# Loop indefinitely for continuous spinning or until max_frames is reached
while True:
    # Rotate the sphere actor around the Z-axis (vertical axis)
    # The `rotateZ` method rotates the actor in place
    sphere.rotateZ(rotation_speed)

    # Rotate all latitude lines around the Z-axis
    for lat_line in plotter.actors[1:1+num_latitudes]: # Skip the sphere, start from first latitude
        lat_line.rotateZ(rotation_speed)

    # Rotate all longitude lines around the Z-axis
    for lon_line in plotter.actors[1+num_latitudes:]: # Start from first longitude
        lon_line.rotateZ(rotation_speed)

    # Render the updated scene
    # This updates the display with the new rotation
    plotter.render()

    # Capture frame for video
    if frame_count < max_frames:
        frame_path = os.path.join(frames_dir, f"frame_{frame_count:04d}.png")
        plotter.screenshot(filename=frame_path)
        frame_count += 1
    else:
        print(f"Captured {max_frames} frames. Creating video...")
        break # Exit loop after capturing enough frames

    # Small delay to control animation speed
    time.sleep(0.02)

    # Check if the plotter window has been closed by the user
    if plotter.escaped:
        print("Plotter window closed by user. Exiting animation.")
        break

# Close the plotter window when the loop finishes (e.g., if user closes it or max_frames reached)
plotter.close()

# --- Video Creation ---
# Collect all captured frame filenames
filenames = [os.path.join(frames_dir, f) for f in sorted(os.listdir(frames_dir)) if f.startswith('frame_') and f.endswith('.png')]

# Create the video from the captured frames
try:
    with imageio.get_writer(video_filename, mode='I', fps=30) as writer: # fps can be adjusted
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
    print(f"Video saved as {video_filename}")
except Exception as e:
    print(f"Error creating video: {e}")

# --- Clean up temporary frame files ---
for filename in filenames:
    os.remove(filename)
os.rmdir(frames_dir)
print(f"Cleaned up temporary frames in {frames_dir} directory.")
